In [ ]:
import pandas as pd
import re

In [ ]:
def make_fullyear(column_list):
    """Function to redefine 'partial year', i.e 'Address99' into 'full year', i.e 'Address1999'.

    :param column_list: list of columns in a given data frame

    :return List of columns with redefined years
    """
    final_columns = ['0'] * len(column_list)
    for i, col in enumerate(column_list):
        # Separate potential year from the rest of the column name
        year = col[-2:]
        col_name = col[:-2]
        if re.match('\d\d', year):  # If the year is digits
            # Figure out century by checking decade
            if year[0] == '9':
                col = '{}{}{}'.format(col_name, '19', year)
            else:
                col = '{}{}{}'.format(col_name, '20', year)
        final_columns[i] = col
    return final_columns

In [ ]:
#just to get a look at columns
address_90_99 = pd.read_table(r"C:\Users\jc4673\Documents\Data\NETS\NETSDatabase2014\NETS2014_AddressSpecial90to99.txt", nrows=10)
address_00_14 = pd.read_table(r"C:\Users\jc4673\Documents\Data\NETS\NETSDatabase2014\NETS2014_AddressSpecial00to14.txt", nrows=10)
sic = pd.read_table(r"C:\Users\jc4673\Documents\Data\NETS\NETSDatabase2014\NETS2014_SIC.txt", nrows=10)
company = pd.read_table(r"C:\Users\jc4673\Documents\Data\NETS\NETSDatabase2014\NETS2014_Company.txt", nrows=10)

In [ ]:
cbsa_1 = [x for x in address_90_99.columns if 'CBSA' in x]
cbsa_2 = [x for x in address_00_14.columns if 'CBSA' in x]
sic_cols = ['DunsNumber'] + [x for x in sic.columns if 'SIC9' in x or 'SIC0' in x or 'SIC1' in x]

In [ ]:
address_90 = pd.read_table(r"C:\Users\jc4673\Documents\Data\NETS\NETSDatabase2014\NETS2014_AddressSpecial90to99.txt",
                              chunksize=10**6, index_col='DunsNumber', encoding='Windows-1252', error_bad_lines=False)
address_00 = pd.read_table(r"C:\Users\jc4673\Documents\Data\NETS\NETSDatabase2014\NETS2014_AddressSpecial00to14.txt",
                              chunksize=10**6, index_col='DunsNumber', encoding='Windows-1252', error_bad_lines=False)
sic = pd.read_table(r"C:\Users\jc4673\Documents\Data\NETS\NETSDatabase2014\NETS2014_SIC.txt",
                    chunksize=10**6, index_col='DunsNumber', usecols=sic_cols)
company = pd.read_table(r"C:\Users\jc4673\Documents\Data\NETS\NETSDatabase2014\NETS2014_Company.txt",
                        chunksize=10**6, index_col='DunsNumber', usecols=['DunsNumber', 'Company', 'TradeName'], quoting=3)

In [ ]:
%%time
ny_filename = r"C:\Users\jc4673\Documents\Data\NETS\for_andrew\ny_nets.csv"
springfield_filename = r"C:\Users\jc4673\Documents\Data\NETS\for_andrew\springfield_nets.csv"

for (ad99_chunk, ad14_chunk, sic_chunk, (num, company_chunk)) in zip(address_90, address_00, sic, enumerate(company)):
    address = pd.concat([ad99_chunk, ad14_chunk], axis=1)
    address_sub = address[cbsa_1 + cbsa_2]
    address_filter_ny = address_sub.apply(lambda x: x.isin([20524, 35004, 35084, 35614]), axis=0).any(axis=1)
    address_filter_spring = address_sub.apply(lambda x: x.isin([25540, 44140]), axis=0).any(axis=1)
    address_filtered_ny = address[address_filter_ny == True]
    address_filtered_spring = address[address_filter_spring == True]
    
    sic_filter = sic_chunk.apply(lambda x: x.isin(range(83220000, 83229999)), axis=0).any(axis=1)
    sic_filtered = sic_chunk[sic_filter == True]
    
    joined_ny = address_filtered_ny.join(sic_filtered, how='inner').join(company_chunk, how='left')
    joined_ny.columns = make_fullyear(joined_ny.columns)
    joined_spring = address_filtered_spring.join(sic_filtered, how='inner').join(company_chunk, how='left')
    joined_spring.columns = make_fullyear(joined_spring.columns)
    
    ny_long = pd.wide_to_long(joined_ny.reset_index(), ['Address', 'City', 'State', 'ZIP', 'CBSA', 'SIC'], i='DunsNumber', j='Year').sort_index()
    ny_long = ny_long[['Company', 'TradeName', 'SIC', 'Address', 'City', 'State', 'ZIP', 'CBSA']].dropna(subset=['SIC'])
    
    spring_long = pd.wide_to_long(joined_spring.reset_index(), ['Address', 'City', 'State', 'ZIP', 'CBSA', 'SIC'], i='DunsNumber', j='Year').sort_index()
    spring_long = spring_long[['Company', 'TradeName', 'SIC', 'Address', 'City', 'State', 'ZIP', 'CBSA']].dropna(subset=['SIC'])
    

    if num == 0:
        ny_long.to_csv(ny_filename)
        spring_long.to_csv(springfield_filename)
    
    else:
        with open(ny_filename, 'a') as nyfile, open(springfield_filename, 'a') as spring_file:
            ny_long.to_csv(nyfile, header=False)
            spring_long.to_csv(spring_file, header=False)
    
    print(num)
